In [ ]:
pip install emoji

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
import emoji
import spacy
import string
from textblob import TextBlob

In [ ]:
df = pd.read_csv('anger.csv')
df.head()

,ID,text,remove,rating
0,10000,How the fu*k! Who the heck! moved my fridge!.....,anger,0.938
1,10001,So my Indian Uber driver just called someone t...,anger,0.896
2,10002,@DPD_UK I asked for my parcel to be delivered ...,anger,0.896
3,10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896
4,10004,Don't join @BTCare they put the phone down on ...,anger,0.896


In [ ]:
df.isnull().sum()

text      0
rating    0
dtype: int64

In [ ]:
df = df.drop(columns = ['ID', 'remove'])

In [ ]:
df.shape

(857, 2)

In [ ]:
#0 Dealing with Chat Words

chat_words = {
    'AFAIK': 'As Far As I Know',
    'AFK': 'Away From Keyboard',
    'ASAP': 'As Soon As Possible',
    'ATK': 'At The Keyboard',
    'ATM': 'At The Moment',
    'A3': 'Anytime, Anywhere, Anyplace',
    'BAK': 'Back At Keyboard',
    'BBL': 'Be Back Later',
    'BBS': 'Be Back Soon',
    'BFN': 'Bye For Now',
    'B4N': 'Bye For Now',
    'BRB': 'Be Right Back',
    'BRT': 'Be Right There',
    'BTW': 'By The Way',
    'B4': 'Before',
    'CU': 'See You',
    'CUL8R': 'See You Later',
    'CYA': 'See You',
    'FAQ': 'Frequently Asked Questions',
    'FC': 'Fingers Crossed',
    'FWIW': 'For What It is Worth',
    'FYI': 'For Your Information',
    'GAL': 'Get A Life',
    'GG': 'Good Game',
    'GN': 'Good Night',
    'GMTA': 'Great Minds Think Alike',
    'GR8': 'Great!',
    'G9': 'Genius',
    'IC': 'I See',
    'ICQ': 'I Seek you',
    'ILU': 'I Love You',
    'IMHO': 'In My Honest Opinion',
    'IMO': 'In My Opinion',
    'IOW': 'In Other Words',
    'IRL': 'In Real Life',
    'KISS': 'Keep It Simple, Stupid',
    'LDR': 'Long Distance Relationship',
    'LMAO': 'Laugh My Ass Off',
    'LOL': 'Laughing Out Loud',
    'LTNS': 'Long Time No See',
    'L8R': 'Later',
    'MTE': 'My Thoughts Exactly',
    'M8': 'Mate',
    'NRN': 'No Reply Necessary',
    'OIC': 'Oh I See',
    'PITA': 'Pain In The Ass',
    'PRT': 'Party',
    'PRW': 'Parents Are Watching',
    'QPSA': 'Que Pasa?',
    'ROFL': 'Rolling On The Floor Laughing',
    'ROFLOL': 'Rolling On The Floor Laughing Out Loud',
    'ROTFLMAO': 'Rolling On The Floor Laughing My A.. Off',
    'SK8': 'Skate',
    'STATS': 'Your sex and age',
    'ASL': 'Age, Sex, Location',
    'THX': 'Thank You',
    'TTFN': 'Ta-Ta For Now!',
    'TTYL': 'Talk To You Later',
    'U': 'You',
    'U2': 'You Too',
    'U4E': 'Yours For Ever',
    'WB': 'Welcome Back',
    'WTF': 'What The Fuck',
    'WTG': 'Way To Go!',
    'WUF': 'Where Are You From?',
    'W8': 'Wait',
    '7K': 'Sick Laugher',
    'TFW': 'That feeling when',
    'MFW': 'My face when',
    'MRW': 'My reaction when',
    'IFYP': 'I feel your pain',
    'LOL': 'Laughing out loud',
    'TNTL': 'Trying not to laugh',
    'JK': 'Just kidding',
    'IDC': 'I do not care',
    'ILY': 'I love you',
    'IMU': 'I miss you',
    'ADIH': 'Another day in hell',
    'ZZZ': 'Sleeping bored tired',
    'WYWH': 'Wish you were here',
    'TIME': 'Tears in my eyes',
    'BAE': 'Before anyone else',
    'FIMH': 'Forever in my heart',
    'BSAAW': 'Big smile and a wink',
    'BWL': 'Bursting with laughter',
    'LMAO': 'Laughing my ass off',
    'BFF': 'Best friends forever',
    'CSL': 'Can not stop laughing',
}

def convert_chat(text):
  new_text = []
  for w in text.split():
    if w.upper() in chat_words:
      new_text.append(chat_words[w.upper()])
    else:
      new_text.append(w)
  return " ".join(new_text)

df['text'] = df['text'].apply(lambda x: convert_chat(x))

In [ ]:
df['text']

0      How the fu*k! Who the heck! moved my fridge!.....
1      So my Indian Uber driver just called someone t...
2      @DPD_UK I asked for my parcel to be delivered ...
3      so ef whichever butt wipe pulled the fire alar...
4      Don't join @BTCare they put the phone down on ...
                             ...                        
852     rose incense are the best thing I've ever bought
853           @jaaames1993 Literally burst out laughing.
854             Follow up. Follow through. Be . #success
855    Wrinkles should merely hide where frown have b...
856    Love the new song I can't stop thinking about ...
Name: text, Length: 857, dtype: object

In [ ]:
#1 Lowercasing

df['text'] = df['text'].str.lower()

In [ ]:
#2 Removing Twitter handles

def remove_twitter_handles(text):
    return re.sub(r'@[A-Za-z0-9_]+', '', text)

df['text'] = df['text'].apply(remove_twitter_handles)

In [ ]:
df['text']

0      how the fu*k! who the heck! moved my fridge!.....
1      so my indian uber driver just called someone t...
2       i asked for my parcel to be delivered to a pi...
3      so ef whichever butt wipe pulled the fire alar...
4      don't join  they put the phone down on you, ta...
                             ...                        
852     rose incense are the best thing i've ever bought
853                        literally burst out laughing.
854             follow up. follow through. be . #success
855    wrinkles should merely hide where frown have b...
856    love the new song i can't stop thinking about ...
Name: text, Length: 857, dtype: object

In [ ]:
#3 Removing URLs (if any)

def remove_url(text):
  pattern = re.compile(r'https://\S+|www\.\S+')
  return pattern.sub(r'', text)

df['text'] = df['text'].apply(lambda x: remove_url(x))

In [ ]:
#4 Removing HTML Tags (if any)

def remove_html(text):

  pattern = re.compile('<.*?>')
  return pattern.sub(r'', text)
df['text'] = df['text'].apply(lambda x: remove_html(x))

In [ ]:
#5 Removing Punctuations

exclude = string.punctuation

def remove_punct1(text):
  return text.translate(str.maketrans('', '', exclude))

df['text'] = df['text'].apply(lambda x: remove_punct1(str(x)))

In [ ]:
#6 Removing white spaces

def preprocess_text(text):
    processed_text = text.strip()
    processed_text = processed_text.replace("  ", " ")

    return processed_text

df['text'] = df['text'].apply(lambda x: preprocess_text(str(x)))

In [ ]:
#7 Removing Stop words

nltk.download('stopwords')

from nltk.corpus import stopwords
# stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def remove_stop_words(text):
  new_text = []

  for word in text.split():
    if word in stopwords.words('english'):
      new_text.append('')
    else:
      new_text.append(word)
  x = new_text[:]
  new_text.clear()
  return " ".join(x)

df['text'] = df['text'].apply(lambda x: remove_stop_words(str(x)))

In [ ]:
#8 Performing Tokenization

import nltk

from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')

word_tokenize((str)(df['text']))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['0',
 'fuk',
 'heck',
 'moved',
 'fridge',
 'knock',
 'landlord',
 'd',
 '...',
 '1',
 'indian',
 'uber',
 'driver',
 'called',
 'someone',
 'n',
 'word',
 '...',
 '2',
 'asked',
 'parcel',
 'delivered',
 'pick',
 'store',
 'ad',
 '...',
 '3',
 'ef',
 'whichever',
 'butt',
 'wipe',
 'pulled',
 'fire',
 'alarm',
 'da',
 '...',
 '4',
 'dont',
 'join',
 'put',
 'phone',
 'talk',
 'rude',
 'taking',
 '...',
 '...',
 '852',
 'rose',
 'incense',
 'best',
 'thing',
 'ive',
 'ever',
 'bought',
 '853',
 'literally',
 'burst',
 'laughing',
 '854',
 'follow',
 'follow',
 'success',
 '855',
 'wrinkles',
 'merely',
 'hide',
 'frown',
 'mark',
 'twain',
 '856',
 'love',
 'new',
 'song',
 'cant',
 'stop',
 'thinking',
 'Name',
 ':',
 'text',
 ',',
 'Length',
 ':',
 '857',
 ',',
 'dtype',
 ':',
 'object']

In [ ]:
#8 POS Tagging

dict1 = spacy.load('en_core_web_sm')
doc = dict1((str)(df['text']))

In [ ]:
for word in doc:
  print(word.text, "------>", word.pos_, word.tag_, spacy.explain(word.tag_))

0 ------> NUM CD cardinal number
        ------> SPACE _SP whitespace
fuk ------> NOUN NN noun, singular or mass
   ------> SPACE _SP whitespace
heck ------> NOUN NN noun, singular or mass
moved ------> VERB VBD verb, past tense
  ------> SPACE _SP whitespace
fridge ------> NOUN NN noun, singular or mass
   ------> SPACE _SP whitespace
knock ------> VERB VBP verb, non-3rd person singular present
  ------> SPACE _SP whitespace
landlord ------> NOUN NN noun, singular or mass
d ------> X LS list item marker
... ------> PUNCT : punctuation mark, colon or ellipsis

 ------> SPACE _SP whitespace
1 ------> NUM CD cardinal number
        ------> SPACE _SP whitespace
indian ------> ADJ JJ adjective (English), other noun-modifier (Chinese)
uber ------> NOUN NN noun, singular or mass
driver ------> NOUN NN noun, singular or mass
  ------> SPACE _SP whitespace
called ------> VERB VBD verb, past tense
someone ------> PRON NN noun, singular or mass
  ------> SPACE _SP whitespace
n ------> NOUN NN no

In [ ]:
df['text']

0        fuk   heck moved  fridge   knock  landlord d...
1        indian uber driver  called someone  n word  ...
2       asked   parcel   delivered   pick  store   ad...
3       ef whichever butt wipe pulled  fire alarm  da...
4      dont join  put  phone    talk     rude taking ...
                             ...                        
852            rose incense   best thing ive ever bought
853                            literally burst  laughing
854                             follow  follow   success
855            wrinkles  merely hide  frown   mark twain
856                love  new song  cant stop thinking   
Name: text, Length: 857, dtype: object

Statistical Model

In [ ]:
import sklearn

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer() #ngram_range = ... for n-grams

bow = cv.fit_transform(df['text'])

In [ ]:
print(cv.vocabulary_)

{'fuk': 1023, 'heck': 1190, 'moved': 1683, 'fridge': 1007, 'knock': 1425, 'landlord': 1442, 'door': 740, 'angry': 159, 'mad': 1564, 'indian': 1294, 'uber': 2662, 'driver': 759, 'called': 418, 'someone': 2342, 'word': 2847, 'wasnt': 2766, 'moving': 1685, 'vehicle': 2710, 'id': 1261, 'jumped': 1385, 'disgusted': 720, 'asked': 205, 'parcel': 1840, 'delivered': 677, 'pick': 1881, 'store': 2406, 'address': 92, 'fuming': 1029, 'poorcustomerservice': 1926, 'ef': 787, 'whichever': 2810, 'butt': 403, 'wipe': 2828, 'pulled': 2009, 'fire': 944, 'alarm': 119, 'davis': 657, 'bc': 267, 'sound': 2356, 'asleep': 207, 'pissed': 1892, 'upset': 2691, 'tired': 2581, 'sad': 2165, 'hangry': 1156, 'dont': 738, 'join': 1370, 'put': 2019, 'phone': 1876, 'talk': 2485, 'rude': 2152, 'taking': 2484, 'money': 1668, 'acc': 68, 'willynilly': 2822, 'blood': 327, 'boiling': 337, 'youve': 2911, 'still': 2399, 'got': 1096, 'whole': 2814, 'season': 2212, 'wentworth': 2796, 'watch': 2771, 'stupid': 2433, 'cunt': 632, 'wor

In [ ]:
print(bow[0].toarray())

[[0 0 0 ... 0 0 0]]


In [ ]:
import numpy as np

In [ ]:
bow.shape

(857, 2937)

In [ ]:
y = df['rating']

In [ ]:
bow.shape

(857, 2937)

In [ ]:
y.shape

(857,)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(bow, y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

mae, mse, rmse, r2

(0.13983639227060343,
 0.031171856925518795,
 0.17655553496143583,
 -0.07069075781133694)